In [1]:
print('Hello wqorld')

Hello wqorld


In [2]:
# Step 1: Install dependencies if not already
# !pip install boto3 requests requests-aws4auth

import boto3
import requests
from requests_aws4auth import AWS4Auth
import json

# --------------------------
# Step 2: AWS session & credentials
# --------------------------
# If your CLI works with default profile, this will pick the same credentials
session = boto3.Session(profile_name="default")  # Change if using another profile
credentials = session.get_credentials().get_frozen_credentials()

region = "us-east-1"  # must match the CLI
service = "aoss"

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

# --------------------------
# Step 3: Get OpenSearch Serverless endpoint
# --------------------------
# You can hardcode your collection endpoint OR dynamically fetch it using boto3
client = session.client("opensearchserverless", region_name=region)

response = client.list_collections()
print("Collections from AOSS:")
print(json.dumps(response, indent=4))

# --------------------------
# Step 4: Optional: Test endpoint connectivity
# --------------------------
# Pick any active collection ID from response
if response.get("collectionSummaries"):
    collection_id = response["collectionSummaries"][0]["id"]
    host = f"https://{collection_id}.{region}.aoss.amazonaws.com"

    # Simple request to check connectivity
    r = requests.get(f"{host}/_cluster/health", auth=awsauth)
    print("\nCluster health status code:", r.status_code)
    print(r.text)


Collections from AOSS:
{
    "collectionSummaries": [
        {
            "id": "5mz690z0qz1ivzufwhml",
            "name": "movies",
            "status": "ACTIVE",
            "arn": "arn:aws:aoss:us-east-1:073914561898:collection/5mz690z0qz1ivzufwhml",
            "kmsKeyArn": "auto"
        }
    ],
    "ResponseMetadata": {
        "RequestId": "ef52b051-e418-4976-9788-b493e75dade6",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "date": "Sat, 31 Jan 2026 04:09:30 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "184",
            "connection": "keep-alive",
            "x-amzn-requestid": "ef52b051-e418-4976-9788-b493e75dade6"
        },
        "RetryAttempts": 0
    }
}

Cluster health status code: 404



In [4]:
import requests
from requests_aws4auth import AWS4Auth
import json

# Use the same awsauth and collection_id from before
collection_id = "5mz690z0qz1ivzufwhml"
region = "us-east-1"
host = f"https://{collection_id}.{region}.aoss.amazonaws.com"

# Example: simple search query
query = {
    "size": 5,
    "query": {
        "match_all": {}
    }
}

r = requests.get(f"{host}/_search", auth=awsauth, json=query)
print("Search response status code:", r.status_code)
print(json.dumps(r.json(), indent=4))


Search response status code: 403
{
    "status": 403,
    "request-id": "465c492c-a222-9f20-a3b5-072fbc72e0da",
    "error": {
        "reason": "403 Forbidden",
        "type": "Forbidden"
    }
}
